<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_and_Manipulated_Media_Analysis_R%26D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Deepfake Detection and Manipulated Media Analysis using Multiagent System and Compound AI Approach**

In [1]:
# Install necessary packages
!pip install -q langchain langchain-community langgraph torch transformers opencv-python librosa numpy face-recognition dlib mediapipe scipy pillow tqdm pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.2/138.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00


In [2]:
import torch
import numpy as np
import cv2
import librosa
import face_recognition
import mediapipe as mp
from typing import Dict, List, Any, Tuple, Optional, Union
from pydantic import BaseModel, Field
from datetime import datetime
import json
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, AutoModelForAudioClassification,
    CLIPProcessor, CLIPModel, VideoMAEFeatureExtractor, VideoMAEForVideoClassification,
    Wav2Vec2Processor, Wav2Vec2ForSequenceClassification, WhisperProcessor, WhisperForAudioClassification,
    BlenderbotTokenizer, BlenderbotForConditionalGeneration, VisionEncoderDecoderModel,
    ViTImageProcessor, InstructBlipProcessor, InstructBlipForConditionalGeneration,
    ImageGPTForCausalImageModeling, TimesformerForVideoClassification,
    BertTokenizer, BertForSequenceClassification,
    T5Tokenizer, T5ForConditionalGeneration,
    RobertaTokenizer, RobertaForSequenceClassification,
    EfficientNetForImageClassification, HubertForCTC, AutoImageProcessor
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.tools import BaseTool
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.schema import AgentAction, AgentFinish
from langgraph.graph import Graph, StateGraph, END
from PIL import Image
import io
import soundfile as sf
import wave
import scipy
import openai
from moviepy.editor import VideoFileClip
import hashlib
from scipy.spatial.distance import cosine
from sklearn.metrics import roc_curve, auc
import re
import os
import asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed

  if event.key is 'enter':



In [3]:
# Pydantic Models
class DeepfakeAnalysisResult(BaseModel):
    score: float = Field(..., description="Confidence score (0-1)")
    label: str = Field(..., description="Classification label")
    anomalies: List[str] = Field(default_factory=list)
    artifacts: List[str] = Field(default_factory=list)
    confidence: float = Field(..., description="Model confidence")
    method: str = Field(..., description="Detection method used")
    timestamp: datetime = Field(default_factory=datetime.now)

class MultimodalAnalysisReport(BaseModel):
    case_id: str
    file_info: Dict[str, Any]
    video_analysis: Optional[DeepfakeAnalysisResult]
    audio_analysis: Optional[DeepfakeAnalysisResult]
    image_analysis: Optional[DeepfakeAnalysisResult]
    text_analysis: Optional[DeepfakeAnalysisResult]
    multimodal_score: float
    verdict: str
    evidence: List[Dict[str, Any]]
    metadata: Dict[str, Any]

In [4]:
# Setup Environment
def setup_enhanced_detection_environment():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    models = {
        "videomae": VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics"),
        "timesformer": TimesformerForVideoClassification.from_pretrained("facebook/timesformer-base-finetuned-k400"),
        "clip": CLIPModel.from_pretrained("openai/clip-vit-large-patch14"),
        "instructblip": InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b"),
        "vit_gpt2": VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning"),
        "efficientnet": EfficientNetForImageClassification.from_pretrained("google/efficientnet-b7"),
        "wav2vec2": Wav2Vec2ForSequenceClassification.from_pretrained("facebook/wav2vec2-large-960h"),
        "whisper": WhisperForAudioClassification.from_pretrained("openai/whisper-large-v3"),
        "hubert": HubertForCTC.from_pretrained("facebook/hubert-large-ls960-ft"),
        "roberta_fake": AutoModelForSequenceClassification.from_pretrained("deepset/roberta-base-squad2"),
        "blenderbot": BlenderbotForConditionalGeneration.from_pretrained("facebook/blenderbot-400M-distill"),
        "bert": BertForSequenceClassification.from_pretrained("bert-base-uncased"),
        "t5": T5ForConditionalGeneration.from_pretrained("t5-base"),
        "face_detector": mp.solutions.face_detection.FaceDetection(min_detection_confidence=0.7),
        "face_mesh": mp.solutions.face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            min_detection_confidence=0.7
        ),
        "processors": {
            "clip": CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14"),
            "instructblip": InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b"),
            "vit": ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k"),
            "wav2vec2": Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h"),
            "whisper": WhisperProcessor.from_pretrained("openai/whisper-large-v3"),
            "roberta": AutoTokenizer.from_pretrained("deepset/roberta-base-squad2"),
            "blenderbot": BlenderbotTokenizer.from_pretrained("facebook/blenderbot-400M-distill"),
            "bert": BertTokenizer.from_pretrained("bert-base-uncased"),
            "t5": T5Tokenizer.from_pretrained("t5-base"),
            "efficientnet": AutoImageProcessor.from_pretrained("google/efficientnet-b7"),
            "hubert": Wav2Vec2Processor.from_pretrained("facebook/hubert-large-ls960-ft")
        },
        "llms": {
            "gpt4": ChatOpenAI(model="gpt-4", temperature=0.2),
            "claude": ChatOpenAI(model="claude-3-opus", temperature=0.2)
        }
    }
    return {"device": device, "models": models}

In [5]:
# Enhanced Preprocessing Techniques
def reduce_noise(audio_data: np.ndarray) -> np.ndarray:
    return librosa.effects.preemphasis(audio_data)

def enhance_resolution(image: np.ndarray) -> np.ndarray:
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    return sr.upsample(image)

def deblur_image(image: np.ndarray) -> np.ndarray:
    # Implement a deblurring algorithm, e.g., Wiener filter
    return cv2.filter2D(image, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))

def extract_audio_features(audio_data: np.ndarray, sample_rate: int) -> Dict[str, Any]:
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sample_rate)
    mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    return {"mfcc": mfcc, "chroma": chroma, "mel": mel}

def extract_image_features(image: np.ndarray) -> Dict[str, Any]:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return {"keypoints": keypoints, "descriptors": descriptors}

def calculate_dense_optical_flow(prev_frame: np.ndarray, curr_frame: np.ndarray) -> np.ndarray:
    gray_prev = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    gray_curr = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray_prev, gray_curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def extract_temporal_features(optical_flow: np.ndarray) -> Dict[str, Any]:
    mag, ang = cv2.cartToPolar(optical_flow[..., 0], optical_flow[..., 1])
    return {"magnitude": mag, "angle": ang}

def estimate_noise(frame: np.ndarray) -> float:
    return np.mean(cv2.Laplacian(frame, cv2.CV_64F).var())

def calculate_contrast(frame: np.ndarray) -> float:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray.std()

def detect_compression_artifacts(frame: np.ndarray) -> float:
    dct = cv2.dct(np.float32(frame)/255.0)
    return np.mean(np.abs(dct))

In [6]:
# Enhanced Preprocessing
async def enhanced_preprocessing(file_path: str) -> Dict[str, Any]:
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension in [".mp3", ".wav", ".flac"]:
        return await enhanced_audio_preprocessing(file_path)
    elif file_extension in [".jpg", ".jpeg", ".png", ".bmp"]:
        return await enhanced_image_preprocessing(file_path)
    elif file_extension in [".mp4", ".avi", ".mov", ".mkv"]:
        return await enhanced_video_preprocessing(file_path)
    else:
        raise ValueError("Unsupported file format")

async def enhanced_audio_preprocessing(audio_path: str) -> Dict[str, Any]:
    audio_data, sample_rate = librosa.load(audio_path, sr=None)
    audio_data = reduce_noise(audio_data)
    audio_features = extract_audio_features(audio_data, sample_rate)
    return {"audio": audio_data, "sample_rate": sample_rate, "audio_features": audio_features}

async def enhanced_image_preprocessing(image_path: str) -> Dict[str, Any]:
    image = cv2.imread(image_path)
    image = enhance_resolution(image)
    image = deblur_image(image)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_features = extract_image_features(image_rgb)
    return {"image": image_rgb, "image_features": image_features}

async def enhanced_video_preprocessing(video_path: str) -> Dict[str, Any]:
    cap = cv2.VideoCapture(video_path)
    frames = []
    audio_data = None
    metadata = {
        "fps": cap.get(cv2.CAP_PROP_FPS),
        "frame_count": int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        "width": int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        "height": int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        "duration": float(cap.get(cv2.CAP_PROP_FRAME_COUNT)) / float(cap.get(cv2.CAP_PROP_FPS)),
        "codec": int(cap.get(cv2.CAP_PROP_FOURCC)).to_bytes(4, byteorder='little').decode(),
        "file_size": os.path.getsize(video_path)
    }
    frame_quality_metrics = []
    optical_flow_data = []
    prev_frame = None
    temporal_features = []

    loop = asyncio.get_event_loop()
    executor = ThreadPoolExecutor(max_workers=4)

    async def process_frame(frame):
        return await loop.run_in_executor(executor, enhance_resolution, frame)

    async def process_quality_metrics(frame):
        return await loop.run_in_executor(executor, lambda: {
            "blur": cv2.Laplacian(frame, cv2.CV_64F).var(),
            "noise": estimate_noise(frame),
            "brightness": np.mean(frame),
            "contrast": calculate_contrast(frame),
            "compression_artifacts": detect_compression_artifacts(frame)
        })

    async def process_optical_flow(prev_frame, frame):
        return await loop.run_in_executor(executor, calculate_dense_optical_flow, prev_frame, frame)

    async def process_temporal_features(flow):
        return await loop.run_in_executor(executor, extract_temporal_features, flow)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame, quality_metrics = await asyncio.gather(
            process_frame(frame),
            process_quality_metrics(frame)
        )
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(rgb_frame)
        frame_quality_metrics.append(quality_metrics)
        if prev_frame is not None:
            flow, temp_features = await asyncio.gather(
                process_optical_flow(prev_frame, frame),
                process_temporal_features(flow)
            )
            optical_flow_data.append(flow)
            temporal_features.append(temp_features)
        prev_frame = frame.copy()
    cap.release()

    try:
        video = VideoFileClip(video_path)
        audio = video.audio
        if audio is not None:
            audio_array = audio.to_soundarray()
            audio_data = extract_audio_features(audio_array, audio.fps)
        video.close()
    except Exception as e:
        print(f"Audio extraction error: {e}")
        audio_data = None

    return {
        "frames": frames,
        "audio": audio_data,
        "metadata": metadata,
        "quality_metrics": frame_quality_metrics,
        "optical_flow": optical_flow_data,
        "temporal_features": temporal_features
    }

In [7]:
# Advanced Analysis Functions
async def advanced_audio_analysis(audio_data: Dict[str, Any], models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    if audio_data is None:
        return DeepfakeAnalysisResult(
            score=0.0,
            label="NO_AUDIO",
            confidence=0.0,
            method="audio_analysis",
            anomalies=["No audio data available"]
        )
    anomalies = []
    scores = []
    wav2vec2_model = models["wav2vec2"].to(device)
    wav2vec2_processor = models["processors"]["wav2vec2"]
    inputs = wav2vec2_processor(
        audio_data["waveform"],
        sampling_rate=audio_data["sample_rate"],
        return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        wav2vec2_output = wav2vec2_model(**inputs)
        wav2vec2_score = torch.softmax(wav2vec2_output.logits, dim=-1)
        scores.append(wav2vec2_score.max().item())
    whisper_model = models["whisper"].to(device)
    whisper_processor = models["processors"]["whisper"]
    inputs = whisper_processor(
        audio_data["waveform"],
        sampling_rate=audio_data["sample_rate"],
        return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        whisper_output = whisper_model(**inputs)
        whisper_score = torch.softmax(whisper_output.logits, dim=-1)
        scores.append(whisper_score.max().item())
    hubert_model = models["hubert"].to(device)
    hubert_processor = models["processors"]["hubert"]
    inputs = hubert_processor(
        audio_data["waveform"],
        sampling_rate=audio_data["sample_rate"],
        return_tensors="pt"
    ).to(device)
    with torch.no_grad():
        hubert_output = hubert_model(**inputs)
        hubert_score = torch.softmax(hubert_output.logits, dim=-1)
        scores.append(hubert_score.max().item())
    spectral_anomalies = analyze_spectral_features(audio_data)
    anomalies.extend(spectral_anomalies)
    temporal_anomalies = analyze_temporal_patterns(audio_data)
    anomalies.extend(temporal_anomalies)
    final_score = np.mean(scores)
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)),
        method="audio_analysis",
        anomalies=anomalies
    )

async def advanced_image_analysis(image: np.ndarray, models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    anomalies = []
    scores = []
    clip_model = models["clip"].to(device)
    clip_processor = models["processors"]["clip"]
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        clip_output = clip_model.get_image_features(**inputs)
        clip_score = analyze_clip_features(clip_output)
        scores.append(clip_score)
    instructblip_model = models["instructblip"].to(device)
    instructblip_processor = models["processors"]["instructblip"]
    prompt = "Analyze this image for signs of digital manipulation or inconsistencies."
    inputs = instructblip_processor(images=image, text=prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = instructblip_model.generate(**inputs)
        analysis = instructblip_processor.decode(outputs[0], skip_special_tokens=True)
        instructblip_score = analyze_instructblip_response(analysis)
        scores.append(instructblip_score)
    efficientnet_model = models["efficientnet"].to(device)
    efficientnet_processor = models["processors"]["efficientnet"]
    inputs = efficientnet_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        efficientnet_output = efficientnet_model(**inputs)
        efficientnet_score = torch.softmax(efficientnet_output.logits, dim=-1)
        scores.append(efficientnet_score.max().item())
    face_anomalies = analyze_face_authenticity(image, models["face_detector"], models["face_mesh"])
    anomalies.extend(face_anomalies)
    quality_metrics = analyze_image_quality(image)
    quality_score = evaluate_quality_metrics(quality_metrics)
    scores.append(quality_score)
    if quality_metrics["compression_level"] > 0.8:
        anomalies.append("High compression artifacts detected")
    if quality_metrics["noise_level"] > 0.7:
        anomalies.append("Suspicious noise patterns detected")
    final_score = np.mean(scores)
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)),
        method="image_analysis",
        anomalies=anomalies
    )

async def advanced_video_analysis(video_data: Dict[str, Any], models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    anomalies = []
    scores = []
    videomae_model = models["videomae"].to(device)
    videomae_processor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
    inputs = videomae_processor(video_data["frames"], return_tensors="pt").to(device)
    with torch.no_grad():
        videomae_output = videomae_model(**inputs)
        videomae_score = torch.softmax(videomae_output.logits, dim=-1)
        scores.append(videomae_score.max().item())
    timesformer_model = models["timesformer"].to(device)
    timesformer_processor = VideoMAEFeatureExtractor.from_pretrained("facebook/timesformer-base-finetuned-k400")
    inputs = timesformer_processor(video_data["frames"], return_tensors="pt").to(device)
    with torch.no_grad():
        timesformer_output = timesformer_model(**inputs)
        timesformer_score = torch.softmax(timesformer_output.logits, dim=-1)
        scores.append(timesformer_score.max().item())
    temporal_anomalies = analyze_temporal_features(video_data["temporal_features"])
    anomalies.extend(temporal_anomalies)
    optical_flow_anomalies = analyze_optical_flow(video_data["optical_flow"])
    anomalies.extend(optical_flow_anomalies)
    final_score = np.mean(scores)
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)),
        method="video_analysis",
        anomalies=anomalies
    )

In [8]:
# Create Deepfake Detection Tools
def create_deepfake_detection_tools(models: Dict[str, Any], device: torch.device) -> List[Tool]:
    tools = [
        Tool(
            name="analyze_video",
            func=lambda x: advanced_video_analysis(x, models, device),
            description="Analyzes video content for signs of manipulation"
        ),
        Tool(
            name="analyze_audio",
            func=lambda x: advanced_audio_analysis(x, models, device),
            description="Analyzes audio content for signs of manipulation"
        ),
        Tool(
            name="analyze_image",
            func=lambda x: advanced_image_analysis(x, models, device),
            description="Analyzes image content for signs of manipulation"
        ),
        Tool(
            name="semantic_analysis",
            func=lambda x: semantic_consistency_analysis(x, models["llms"]),
            description="Analyzes semantic consistency across modalities"
        ),
        Tool(
            name="face_forgery_detection",
            func=lambda x: face_forgery_detection(x, models, device),
            description="Detects face forgeries in video content"
        ),
        Tool(
            name="lip_sync_detection",
            func=lambda x: lip_sync_detection(x, models, device),
            description="Analyzes lip-sync consistency between audio and video"
        ),
        Tool(
            name="background_consistency",
            func=lambda x: background_consistency_analysis(x, models, device),
            description="Analyzes background consistency across frames"
        ),
        Tool(
            name="real_time_streaming_analysis",
            func=lambda x: real_time_streaming_analysis(x, models, device),
            description="Analyzes live video streams for signs of manipulation"
        ),
        Tool(
            name="text_analysis",
            func=lambda x: text_analysis(x, models["llms"]),
            description="Analyzes text content for signs of manipulation"
        ),
        Tool(
            name="metadata_analysis",
            func=lambda x: metadata_analysis(x),
            description="Analyzes metadata for signs of manipulation"
        )
    ]
    return tools

In [9]:
# Create Detection Agent
def create_detection_agent(tools: List[Tool], llm: ChatOpenAI) -> AgentExecutor:
    class DeepfakeDetectionOutputParser:
        def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
            try:
                if "Final Answer:" in llm_output:
                    return AgentFinish(
                        return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                        log=llm_output,
                    )
                action_match = re.search(r"Action: (.*?)\nAction Input: (.*)", llm_output, re.DOTALL)
                if not action_match:
                    raise ValueError("Could not parse LLM output: " + llm_output)
                action = action_match.group(1).strip()
                action_input = action_match.group(2).strip()
                return AgentAction(tool=action, tool_input=action_input, log=llm_output)
            except Exception as e:
                raise ValueError(f"Could not parse LLM output: {llm_output}") from e
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template("""
            You are an expert deepfake detection system. Your goal is to analyze content across multiple modalities
            to determine authenticity. Consider all available evidence and patterns including:

            1. Visual elements: inconsistencies, artifacts, unnatural patterns
            2. Audio characteristics: synthetic artifacts, unnatural transitions
            3. Semantic coherence: logical consistency across modalities
            4. Temporal patterns: synchronization, continuity
            5. Biometric features: facial landmarks, expressions, movements

            Available tools:
            {tools}

            Process:
            1. Analyze the input using appropriate tools
            2. Evaluate evidence across modalities
            3. Make a final determination on authenticity

            Format your response as:
            Action: [tool name]
            Action Input: [tool input]
            Observation: [result]
            ... (repeat for additional tools as needed)
            Final Answer: [detailed analysis and verdict]
            """),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    return AgentExecutor.from_agent_and_tools(
        agent=LLMSingleActionAgent(
            llm_chain=LLMChain(llm=llm, prompt=prompt),
            output_parser=DeepfakeDetectionOutputParser(),
            stop=["Observation:", "Final Answer:"],
            allowed_tools=[tool.name for tool in tools]
        ),
        tools=tools,
        verbose=True
    )

# Create Detection Graph
def create_detection_graph() -> StateGraph:
    def preprocess(state):
        input_data = state["input"]
        processed_data = asyncio.run(enhanced_preprocessing(input_data))
        return {**state, "processed_data": processed_data}
    def analyze_modalities(state):
        processed_data = state["processed_data"]
        models = state["models"]
        device = state["device"]
        results = {
            "video": asyncio.run(advanced_video_analysis(processed_data, models, device)) if "frames" in processed_data else None,
            "audio": asyncio.run(advanced_audio_analysis(processed_data.get("audio"), models, device)) if "audio" in processed_data else None,
            "image": asyncio.run(advanced_image_analysis(processed_data["image"], models, device)) if "image" in processed_data else None,
            "text": asyncio.run(text_analysis(processed_data.get("text"), models["llms"])) if "text" in processed_data else None
        }
        return {**state, "modality_results": results}
    def cross_modal_analysis(state):
        results = state["modality_results"]
        processed_data = state["processed_data"]
        models = state["models"]
        cross_modal_score = analyze_cross_modal_consistency(results, processed_data, models)
        return {**state, "cross_modal_score": cross_modal_score}
    def generate_report(state):
        results = state["modality_results"]
        cross_modal_score = state["cross_modal_score"]
        processed_data = state["processed_data"]
        report = generate_comprehensive_report(results, cross_modal_score, processed_data)
        return {**state, "final_report": report}
    workflow = StateGraph(nodes=[
        ("preprocess", preprocess),
        ("analyze_modalities", analyze_modalities),
        ("cross_modal_analysis", cross_modal_analysis),
        ("generate_report", generate_report)
    ])
    workflow.add_edge("preprocess", "analyze_modalities")
    workflow.add_edge("analyze_modalities", "cross_modal_analysis")
    workflow.add_edge("cross_modal_analysis", "generate_report")
    workflow.add_edge("generate_report", END)
    return workflow

In [10]:
# Analyze Cross-Modal Consistency
def analyze_cross_modal_consistency(
    results: Dict[str, DeepfakeAnalysisResult],
    processed_data: Dict[str, Any],
    models: Dict[str, Any]
) -> float:
    scores = []
    if results["audio"] and results["video"]:
        sync_score = analyze_av_sync(processed_data["frames"], processed_data["audio"])
        scores.append(sync_score)
    semantic_score = analyze_semantic_consistency(results, processed_data, models["llms"])
    scores.append(semantic_score)
    temporal_score = analyze_temporal_coherence(processed_data["temporal_features"])
    scores.append(temporal_score)
    bio_score = analyze_biometric_consistency(processed_data["frames"], models)
    scores.append(bio_score)
    return float(np.mean(scores))

# Analyze Semantic Consistency
def analyze_semantic_consistency(
    results: Dict[str, DeepfakeAnalysisResult],
    processed_data: Dict[str, Any],
    llms: Dict[str, ChatOpenAI]
) -> float:
    prompt = ChatPromptTemplate.from_template("""
        Analyze the consistency between different modalities in the content:

        Video Analysis: {video_analysis}
        Audio Analysis: {audio_analysis}
        Image Analysis: {image_analysis}
        Text Analysis: {text_analysis}

        Consider:
        1. Do the modalities tell a coherent story?
        2. Are there logical contradictions?
        3. Do temporal patterns align?
        4. Is the emotional content consistent?

        Rate the consistency from 0 to 1, where 1 is perfectly consistent.
        Provide detailed reasoning for your rating.

        Output format:
        Score: [0-1]
        Reasoning: [detailed explanation]
    """)
    chain = LLMChain(llm=llms["gpt4"], prompt=prompt)
    response = chain.run({
        "video_analysis": results["video"].dict() if results["video"] else "N/A",
        "audio_analysis": results["audio"].dict() if results["audio"] else "N/A",
        "image_analysis": results["image"].dict() if results["image"] else "N/A",
        "text_analysis": results["text"].dict() if results["text"] else "N/A"
    })
    score_match = re.search(r"Score: (0\.\d+|1\.0)", response)
    if score_match:
        return float(score_match.group(1))
    return 0.5

In [11]:
# Generate Comprehensive Report
def generate_comprehensive_report(
    results: Dict[str, DeepfakeAnalysisResult],
    cross_modal_score: float,
    processed_data: Dict[str, Any]
) -> MultimodalAnalysisReport:
    scores = [
        results["video"].score if results["video"] else 0.5,
        results["audio"].score if results["audio"] else 0.5,
        results["image"].score if results["image"] else 0.5,
        results["text"].score if results["text"] else 0.5,
        cross_modal_score
    ]
    weights = [0.3, 0.2, 0.2, 0.2, 0.1]
    final_score = sum(s * w for s, w in zip(scores, weights))
    evidence = []
    for modality, result in results.items():
        if result:
            evidence.extend([
                {
                    "type": modality,
                    "description": anomaly,
                    "confidence": result.confidence,
                    "method": result.method
                }
                for anomaly in result.anomalies
            ])
    return MultimodalAnalysisReport(
        case_id=f"DFD-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
        file_info=processed_data["metadata"] if "metadata" in processed_data else {},
        video_analysis=results["video"],
        audio_analysis=results["audio"],
        image_analysis=results["image"],
        text_analysis=results["text"],
        multimodal_score=float(final_score),
        verdict="AUTHENTIC" if final_score > 0.7 else "MANIPULATED",
        evidence=evidence,
        metadata={
            "processing_time": datetime.now().isoformat(),
            "models_used": list(results.keys()),
            "cross_modal_score": cross_modal_score,
            "confidence_distribution": {
                modality: result.confidence
                for modality, result in results.items() if result
            }
        }
    )

# Real-Time Streaming Analysis
async def real_time_streaming_analysis(video_stream: Any, models: Dict[str, Any], device: torch.device) -> DeepfakeAnalysisResult:
    frames = []
    audio_data = None
    metadata = {
        "fps": 30,  # Assuming 30 FPS for real-time streaming
        "frame_count": 0,
        "width": 0,
        "height": 0,
        "duration": 0,
        "codec": "N/A",
        "file_size": 0
    }
    frame_quality_metrics = []
    optical_flow_data = []
    prev_frame = None
    temporal_features = []

    loop = asyncio.get_event_loop()
    executor = ThreadPoolExecutor(max_workers=4)

    async def process_frame(frame):
        return await loop.run_in_executor(executor, enhance_resolution, frame)

    async def process_quality_metrics(frame):
        return await loop.run_in_executor(executor, lambda: {
            "blur": cv2.Laplacian(frame, cv2.CV_64F).var(),
            "noise": estimate_noise(frame),
            "brightness": np.mean(frame),
            "contrast": calculate_contrast(frame),
            "compression_artifacts": detect_compression_artifacts(frame)
        })

    async def process_optical_flow(prev_frame, frame):
        return await loop.run_in_executor(executor, calculate_dense_optical_flow, prev_frame, frame)

    async def process_temporal_features(flow):
        return await loop.run_in_executor(executor, extract_temporal_features, flow)

    while True:
        frame = await video_stream.read()
        if frame is None:
            break
        frame, quality_metrics = await asyncio.gather(
            process_frame(frame),
            process_quality_metrics(frame)
        )
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(rgb_frame)
        frame_quality_metrics.append(quality_metrics)
        if prev_frame is not None:
            flow, temp_features = await asyncio.gather(
                process_optical_flow(prev_frame, frame),
                process_temporal_features(flow)
            )
            optical_flow_data.append(flow)
            temporal_features.append(temp_features)
        prev_frame = frame.copy()

    video_data = {
        "frames": frames,
        "audio": audio_data,
        "metadata": metadata,
        "quality_metrics": frame_quality_metrics,
        "optical_flow": optical_flow_data,
        "temporal_features": temporal_features
    }

    return await advanced_video_analysis(video_data, models, device)

# Text Analysis
async def text_analysis(text: str, llms: Dict[str, ChatOpenAI]) -> DeepfakeAnalysisResult:
    prompt = ChatPromptTemplate.from_template("""
        Analyze the following text for signs of manipulation or inconsistencies:

        Text: {text}

        Consider:
        1. Logical consistency
        2. Temporal coherence
        3. Emotional content
        4. Syntactic patterns

        Rate the authenticity from 0 to 1, where 1 is perfectly authentic.
        Provide detailed reasoning for your rating.

        Output format:
        Score: [0-1]
        Reasoning: [detailed explanation]
    """)
    chain = LLMChain(llm=llms["gpt4"], prompt=prompt)
    response = chain.run({"text": text})
    score_match = re.search(r"Score: (0\.\d+|1\.0)", response)
    if score_match:
        score = float(score_match.group(1))
    else:
        score = 0.5
    anomalies = [line.strip() for line in response.split("Reasoning:")[-1].strip().split("\n") if line.strip()]
    return DeepfakeAnalysisResult(
        score=score,
        label="REAL" if score > 0.7 else "FAKE",
        confidence=0.0,  # Confidence is not applicable for text analysis
        method="text_analysis",
        anomalies=anomalies
    )

# Metadata Analysis
def metadata_analysis(metadata: Dict[str, Any]) -> DeepfakeAnalysisResult:
    anomalies = []
    scores = []

    # Example metadata analysis
    if metadata.get("fps") < 10:
        anomalies.append("Unusually low frame rate")
        scores.append(0.2)
    if metadata.get("duration") < 1:
        anomalies.append("Unusually short duration")
        scores.append(0.2)
    if metadata.get("file_size") < 100000:
        anomalies.append("Unusually small file size")
        scores.append(0.2)

    final_score = np.mean(scores) if scores else 0.5
    return DeepfakeAnalysisResult(
        score=float(final_score),
        label="REAL" if final_score > 0.7 else "FAKE",
        confidence=float(np.std(scores)) if scores else 0.0,
        method="metadata_analysis",
        anomalies=anomalies
    )

# Run Deepfake Detection
def run_deepfake_detection(file_path: str, mode: str = "all") -> MultimodalAnalysisReport:
    try:
        env = setup_enhanced_detection_environment()
        tools = create_deepfake_detection_tools(env["models"], env["device"])
        agent = create_detection_agent(tools, env["models"]["llms"]["gpt4"])
        workflow = create_detection_graph()
        initial_state = {
            "input": file_path,
            "mode": mode,
            "models": env["models"],
            "device": env["device"]
        }
        final_state = workflow.run(initial_state)
        return final_state["final_report"]
    except Exception as e:
        print(f"Error in deepfake detection: {str(e)}")
        raise

In [ ]:
if __name__ == "__main__":
    file_path = "path/to/your/content.mp4"
    report = asyncio.run(run_deepfake_detection(file_path))
    print("\nDeepfake Detection Report:")
    print(json.dumps(report.dict(), indent=2, default=str))

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/22.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/22.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/486M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]